In [17]:
import tensorflow as tf
from sklearn.datasets import load_diabetes

X_diab, y_diab = load_diabetes(return_X_y=True)

In [34]:
class Layer:
    def __init__(self, units, input_layer=False):
        self.input_layer = input_layer
        self.units = units
        self.activation = 'relu'
        self.grad_w = None
        self.grad_b = None

    @tf.function
    def activationFunction(self, z):
        if self.activation == "linear":
            return z

        if self.activation == "relu":
            return tf.maximum(z, tf.zeros(z.shape))

        if self.activation == "sigmoid":
            return 1 / (1 + tf.exp(-z))

    def _setGrad(self, w, b):
        self.grad_w = w
        self.grad_b = b

    def _updateGrad(self):
        self.w.assign_sub(0.0001 * self.grad_w)
        self.bias.assign_sub(0.0001 * self.grad_b)

    def _weightInit(self, input_size):
        self.w = tf.Variable(tf.random.normal((input_size, self.units), mean = 0, stddev = 1 / input_size, name = 'w'))
        self.bias = tf.Variable(tf.zeros((1, self.units)))

    @tf.function
    def __call__(self, X):
        return self.activationFunction(X @ self.w + self.bias)

In [35]:
class DenseNetwork:
    def __init__(self, layers):
        self.layers = layers

        for i in range(1, len(self.layers)):
            self.layers[i]._weightInit(self.layers[i - 1].units)


    def fit(self, X, y):
        X = tf.constant(X, dtype="float32")
        y = tf.constant(y, dtype="float32")
        
        for _ in range(100):
            with tf.GradientTape(persistent=True) as tape:
                loss = self.network(X, y)

            for layer in self.layers[1:]:
                layer._setGrad(tape.gradient(loss, layer.w), tape.gradient(loss, layer.bias))

            for layer in self.layers[1:]:
                layer._updateGrad()
                layer._updateGrad()
                
        del tape

    def predict(self, X):
        X = tf.constant(X, dtype='float32')
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    @tf.function
    def network(self, X, y):
        y_pred = self.forward(X)
        loss = tf.losses.mean_squared_error(y_pred, y)
        tf.print(loss)
        return loss

    @tf.function
    def forward(self, X):
        for layer in self.layers[1:]:
            X = layer(X)
        return X

In [36]:
nn = DenseNetwork([Layer(10), Layer(30),  Layer(1)])

nn.fit(X_diab, y_diab)

nn.predict(X_diab)

[29073.9941 29074.1289 29074.1953 ... 29074.2656 29073.748 29074.3242]
tf.Tensor(
[[-1155.2963 ]
 [ -499.30423]
 [ -638.6827 ]
 [ -363.91232]
 [ -882.5522 ]
 [ -392.44003]
 [ -459.75702]
 [ -472.56674]
 [ -260.08368]
 [ -401.71478]
 [ -624.6225 ]
 [ -857.8601 ]
 [ -438.23694]
 [ -683.0152 ]
 [ -532.8904 ]
 [-1002.88525]
 [-1040.138  ]
 [ -661.4101 ]
 [ -650.71826]
 [ -746.38824]
 [ -523.95154]
 [ -889.8892 ]
 [ -822.8714 ]
 [ -373.8445 ]
 [ -721.4624 ]
 [ -532.0944 ]
 [ -629.72046]
 [ -786.93726]
 [-1532.7142 ]
 [ -364.91696]], shape=(30, 1), dtype=float32)
[22519.7734 22523.4863 22522.7422 ... 22524.1113 22516.8672 22527.2852]
tf.Tensor(
[[      0.     ]
 [  -6256.725  ]
 [   -387.47186]
 [      0.     ]
 [ -23176.191  ]
 [ -35577.996  ]
 [      0.     ]
 [ -16657.44   ]
 [ -13019.048  ]
 [      0.     ]
 [      0.     ]
 [ -26717.398  ]
 [ -52849.926  ]
 [      0.     ]
 [      0.     ]
 [ -44415.004  ]
 [      0.     ]
 [-130171.586  ]
 [ -12433.125  ]
 [ -89693.49   ]
 [  -3422.947

<tf.Tensor: shape=(442, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
 